# Debugging autoreload

In [1]:
%load_ext autoreload
%autoreload 2

# Load packages

In [1]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re

def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

def form_bar(base):
    def formatter(x):
        return f'{str(int(round(x * base)))}/{base}'
    return formatter


# Load data

In [57]:
path = f"D:/YandexDisk/Work/bbd/atlas"

data_raw = pd.read_excel(f"{path}/Биологический возраст.xlsx", skiprows=6)

data_raw['Age'] = pd.to_numeric(data_raw['Возраст пациента'].str.split(' ').str[0])
data_raw['Sex'] = data_raw['Пол пациента'].replace({'жен': 'F', 'муж': 'M'})
for f in['Рост', 'Вес', 'ИМТ']:
    data_raw[f] = data_raw[f].astype(str).str.replace(',', '.')
data_raw[['Рост', 'Вес', 'ИМТ']] = data_raw[['Рост', 'Вес', 'ИМТ']].apply(pd.to_numeric, errors='coerce', axis=1)
hrs = data_raw['ЧСС'].astype(str).str.extract(r"^(\d+)\D*(\d*)$")
hrs = hrs.apply(pd.to_numeric, errors='coerce', axis=1)
hrs['HR'] = hrs[0]
hrs.loc[hrs[1].notna(), 'HR'] = 0.5 * (hrs.loc[hrs[1].notna(), 0] + hrs.loc[hrs[1].notna(), 1])
data_raw['Heart rate'] = hrs['HR']
data_raw['Единицы измерения'] = data_raw['Единицы измерения'].replace({'%': np.nan})
data_raw['Feature'] = data_raw['Название параметра'].astype(str) + ' ' + data_raw['Единицы измерения'].astype(str).replace({'nan': ''})
data_raw.loc[data_raw['Единицы измерения'].notna(), 'Feature'] = data_raw.loc[data_raw['Единицы измерения'].notna(), 'Название параметра'].astype(str) + ', ' + data_raw.loc[data_raw['Единицы измерения'].notna(),'Единицы измерения'].astype(str).replace({'nan': ''})

data = data_raw.pivot(index='№ карты', columns='Feature', values='Значение параметра')
data = data.apply(pd.to_numeric, errors='coerce', axis=1)

feats_add = ['Age', 'Sex', 'Рост', 'Вес', 'ИМТ', 'Heart rate']
for sample_id in data.index.values:
    for f in feats_add:
        f_vals = data_raw.loc[data_raw['№ карты'] == sample_id, f].unique()
        if len(f_vals) > 1:
            print(f_vals)
        else:
            data.at[sample_id, f] = f_vals[0]

In [73]:
nan_feats = data.isna().sum(axis=0).to_frame(name="Number of NaNs")
nan_feats["% of NaNs"] = nan_feats["Number of NaNs"] / data.shape[0] * 100
nan_feats["Number of not-NaNs"] = data.notna().sum(axis=0)
nan_feats.sort_values(["% of NaNs"], ascending=[False], inplace=True)
nan_feats.to_excel(f"{path}/nan_feats.xlsx", index_label="Features")

sns.set_theme(style='ticks')
plt.figure(figsize=(4, 25))
barplot = sns.barplot(
    data=nan_feats,
    x=f"% of NaNs",
    y=nan_feats.index,
    edgecolor='black',
    dodge=False,
)
for container in barplot.containers:
    barplot.bar_label(container, label_type='edge', fmt='%.2f', fontsize=6, padding=2.5)
plt.savefig(f"{path}/feats_barplot.png", bbox_inches='tight', dpi=200)
plt.savefig(f"{path}/feats_barplot.pdf", bbox_inches='tight')
plt.close()
